<a href="https://colab.research.google.com/github/Feldt/DeepLearning_NoteBook/blob/main/CIFAR10_Autoencoder_CNN_BaselineReto_SinSolucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
# from keras.utils import np_utils
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

2025-06-09 11:26:13.637945: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Cargar el dataset cifar10 en un conjunto inicial de training (imagenes y labels) y test (imágenes y labels)

In [2]:
(x_train0, y_train0), (x_test0, y_test0) = datasets.cifar10.load_data()


In [3]:
# Creamos la lista de etiquetas CIFAR10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### 1. Pre-procesamiento del dataset

a. Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

b. Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

c. Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


In [4]:
# Convierte los pixeles de las imágenes del conjunto de train y test a tipo float
x_train = x_train0.astype('float32')
x_test = x_test0.astype('float32')
# Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.
x_train  = x_train / 255
x_test = x_test / 255

# Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical
y_train = keras.utils.to_categorical(y_train0, 10)
y_test = keras.utils.to_categorical(y_test0,10)

Dividimos el test set en 7000 imagenes de validacion (x_val_images) y 3000 de test set (x_test_images)


In [5]:
# Renombrar por claridad
x_train_images = x_train
x_test_images = x_test


In [6]:
# Dividir test en validacion y test, 7000 de validacion y 3000 de test
x_val_images = x_test_images[:7000]
x_test_images = x_test_images[7000:]

Dividimos las etiquetas de validacion (y_val y y_test) de tal forma que correspondan a los conjuntos previos.

In [7]:
# Dividir etiquetas de validacion
y_val_labels = y_test[:7000]
y_test_labels = y_test[7000:]

In [8]:
# Algunos imports complementarios
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, MaxPool2D, Flatten, BatchNormalization
import keras.backend as K
from keras.models import Model

In [9]:

# Modelo encoder-decoder de referencia

input_img = Input(shape=(32, 32, 3))

# Red encoder
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Red decoder
x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [10]:
# Modelo Encoder-Decoder
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 16)       │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 4, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 52,975 (206.93 KB)

 Trainable params: 52,521 (205.16 KB)

 Non-trainable params: 454 (1.77 KB)

### 2. Entrenamiento de autoencoder

a. Entrena el modelo no-supervisado encoder-decoder de tal forma que el modelo aprenda a reconstruir su propio input, con batch_size 64, mínimo 20 épocas.

b. Investiga e implementa el uso de los callbacks de EarlyStopping y ModelCheckpoint

In [11]:

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_autoencoder.h5', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x_train_images, x_train_images,
                    epochs=20,
                    batch_size=64,
                    shuffle=True,
                    validation_data=(x_val_images, x_val_images),
                    callbacks=[early_stop, checkpoint])


2025-06-09 11:27:02.780642: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/20


2025-06-09 11:27:06.530840: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 711ms/step - loss: 0.6000

782/782 ━━━━━━━━━━━━━━━━━━━━ 612s 737ms/step - loss: 0.5999 - val_loss: 0.5781
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 616ms/step - loss: 0.5751

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 464s 593ms/step - loss: 0.5643 - val_loss: 0.5649
Epoch 13/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 576ms/step - loss: 0.5647

782/782 ━━━━━━━━━━━━━━━━━━━━ 464s 593ms/step - loss: 0.5647 - val_loss: 0.5646
Epoch 14/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - loss: 0.5641

782/782 ━━━━━━━━━━━━━━━━━━━━ 506s 647ms/step - loss: 0.5641 - val_loss: 0.5646
Epoch 15/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 657ms/step - loss: 0.5634

782/782 ━━━━━━━━━━━━━━━━━━━━ 526s 673ms/step - loss: 0.5634 - val_loss: 0.5642
Epoch 16/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 456s 583ms/step - loss: 0.5637 - val_loss: 0.5643
Epoch 17/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 566ms/step - loss: 0.5634

782/782 ━━━━━━━━━━━━━━━━━━━━ 456s 583ms/step - loss: 0.5634 - val_loss: 0.5642
Epoch 18/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 456s 583ms/step - loss: 0.5636 - val_loss: 0.5643
Epoch 19/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 456s 583ms/step - loss: 0.5639 - val_loss: 0.5650
Epoch 20/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 464s 594ms/step - loss: 0.5634 - val_loss: 0.5654


c. Visualización del proceso de entrenamiento. Grafica Training Loss vs. Validation Loss.


In [ ]:
# Gráfica de Training Loss vs Validation Loss
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Época')
plt.ylabel('Pérdida (Loss)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

d. Visualización del input reconstruido

Imprime un conjunto de imágenes originales y comparalas con la imagen reconstruida por el autoencoder.

Utiliza las siguientes funciones de referencia.

In [ ]:
c10test = model.predict(x_test_images)
c10val = model.predict(x_val_images)

In [ ]:
# Funcion para mostrar imagenes originales y reconstruidas

def showOrigDec(orig, dec, num=10):
    import matplotlib.pyplot as plt
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # reconstruidas
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


In [ ]:
# Muestra algunas imagenes originales y reconstruidas utilizando tu funcion
showOrigDec(x_test_images, c10test)


A continuación generaremos el modelo encoder del autoencoder. Este modelo lo utilizarás para obtener la representación codificada de los conjuntos originales y con ella entrenar un clasificador.

In [ ]:
# Modelo Encoder
encoder = Model(input_img, encoded)
encoder.summary()

### 3. Extracción de features del autoencoder

Utilizando el **método predict del modelo encoder** extrae las variables que te indicamos a continuación:

a. Codifica el el conjunto de imágenes de entrenamiento utilizando el método predict del encoder, y guárdalo en una variable llamada **gist_train_ae**

b. Codifica el conjunto de imágenes de validación, utilizando el método predict del encoder y guárdalo en una variable llamada **gist_valid_ae**

c. Codifica el conjunto de imágenes de prueba utilizando el método predict del encoder y guárdalo en una variable llamada **gist_test_ae**


In [ ]:
# Completa esta función...
gist_train_ae = encoder.predict(x_train0)
gist_valid_ae = encoder.predict(x_val_images)
gist_test_ae = encoder.predict(x_test_images)

A continuación definimos un clasificador con una capa convolucional y dos capas densas, que aprenderá a clasificar el input una vez procesado por el codificador.

Puedes utilizar el siguiente clasificador como referencia:

In [ ]:
num_classes = 10
input = Input((gist_train_ae.shape[1], gist_train_ae.shape[2], gist_train_ae.shape[3]))

x = Conv2D(64, 3, padding="same")(input)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

decoder_classifier = Model(input, output)
decoder_classifier.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])
decoder_classifier.summary()

### 4. Entrenamiento y evaluación del clasificador

a. Entrena el clasificador **decoder_classifier** definido en la fase previa utilizando como input la representación codificada del training set obtenida en el paso previo y como output los labels originales del conjunto de training.

b. Calcula la pérdida de validación del modelo, utilizando la representación codificada de los datos de validación y como ouput los labels originales del conjunto de validación.


In [ ]:


early_stop_clf = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint_clf = ModelCheckpoint('best_decoder_classifier.h5', save_best_only=True, monitor='val_loss', mode='min')

callbacks = [early_stop_clf, checkpoint_clf]

decoder_classifier.fit(gist_train_ae, y_train,
                       validation_data=(gist_valid_ae, y_val_labels),
                       batch_size=64, epochs=20, callbacks=callbacks)

c. Genera predicciones con el modelo clasificador, utiliza el conjunto de test codificado modelo encoder. Guárdalo en la variable pred.




In [ ]:
# Obtener las etiquetas con el clasificador
pred = decoder_classifier.predict(gist_test_ae)
print(pred)
print()
print()
print()

# Convertimos las predicciones a una lista de etiquetas única
pred_classes = np.argmax(pred, axis=1)
print(pred_classes)


d. Evalúa las predicciones del modelo y obten la matriz de confusión.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

cm = confusion_matrix(np.argmax(y_test_labels, axis=1), pred_classes)
print("Matriz de confusión:\n")
print(cm)
print("Classification Report:\n")
cr=classification_report(y_test_labels.argmax(axis=1), pred.argmax(axis=1), target_names=class_names)
print(cr)

### 5. Mejoras a los modelos

Aplicando los conceptos del curso modifica dichas redes para obtener un mejor accuracy ¿Crees poder lograr un 70% o tal vez 80% de accuracy con tu modelo? OJO: NO está permitido modificar el modelo clasificador.

a.    Experimenta agregando capas, modificando operaciones y modificando las dimensiones de las capas actuales. OJO: Recuerda que para que tu modelo encoder-decoder siga funcionando y puedas reconstruir las imágenes codificadas, las capas de MaxPool del encoder deben de corresponder a las capas UpSample del decoder. Tip: ¿Las capas de pooling ayudan o perjudican a tu modelo?


b.    En una celda de texto, justifica los cambios realizados, a la arquitectura.


c.     Genera la matriz de confusión de tu ensamble de modelos mejorado. Recuerda que debes re-entrenar el clasificador si la arquitectura del autoencoder cambia.


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# -------------------- AUTOENCODER MEJORADO --------------------
input_img = Input(shape=(32, 32, 3))

x = Conv2D(128, (3, 3), padding='same')(input_img)               # Mejora 1: más filtros
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (3, 3), strides=2, padding='same')(x)             # Mejora 2: reducir tamaño sin perder info
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (3, 3), strides=2, padding='same')(x)             # Mejora 3: compactación progresiva
x = BatchNormalization()(x)
x = Activation('relu')(x)

encoded = Conv2D(32, (3, 3), strides=2, padding='same')(x)       # Mejora 4: espacio latente rico
encoded = BatchNormalization()(encoded)
encoded = Activation('relu')(encoded)

x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# -------------------- ENTRENAMIENTO AUTOENCODER --------------------
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_autoencoder_v2.h5', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x_train, x_train,
                    epochs=40,
                    batch_size=64,
                    shuffle=True,
                    validation_data=(x_val_images, x_val_images),
                    callbacks=[early_stop, checkpoint])

# -------------------- EXTRACCIÓN DE FEATURES --------------------
encoder = Model(input_img, encoded)
gist_train_ae = encoder.predict(x_train)
gist_valid_ae = encoder.predict(x_val_images)
gist_test_ae = encoder.predict(x_test_images)

# -------------------- REENTRENAR CLASIFICADOR EXISTENTE --------------------
decoder_classifier.fit(gist_train_ae, y_train, 
                       validation_data=(gist_valid_ae, y_val_labels),
                       batch_size=64,
                       epochs=20,
                       callbacks=[early_stop, checkpoint])

# -------------------- EVALUACIÓN FINAL --------------------
pred = decoder_classifier.predict(gist_test_ae)
pred_classes = np.argmax(pred, axis=1)

cm = confusion_matrix(np.argmax(y_test_labels, axis=1), pred_classes)
cr = classification_report(np.argmax(y_test_labels, axis=1), pred_classes, target_names=class_names)

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", cr)
